### BASE MODEL TO DETECT PARASITES and NON_PARASITES

* The data set is downloaded from [kaggle](https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria).
* In this notebook we are building a model and saving it to be used later for Malaria stage detection. 
* `VGG-16` architecture is used here.
* We are training last three `feature` network and a new `classifier` network.
* `Adam` optimiser is used for the feature network and `SGD` is used for the classifier network.
* The learning rate for both the optimisiers are decayed per 4 epochs by 0.1.

In [ ]:
# Import the required modules
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import time
import torchvision
import copy
from torch.utils.data import random_split
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
# from torch.utils.data.sampler import SubsetRandomSampler

torch.cuda.current_device() # Work around for the Bug https://github.com/pytorch/pytorch/issues/20635

In [ ]:
# Dataset path
data_dir = r"C:\ADM_project\cell_images"

In [ ]:
# Transformations for each type of the dataset
transformormations = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(50),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(240),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(240),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

dataset = { x : datasets.ImageFolder(os.path.join(data_dir, x), transformormations[x])
               for x in ['train', 'test', 'valid']
          }

dataset_loaders = {x : torch.utils.data.DataLoader(dataset[x], batch_size=16,
                        shuffle=True, num_workers=4) for x in ['train', 'test', 'valid']
                  }

In [ ]:
# Assign the device for computation
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
# As a pre-trained model we are considering VGG-16 with the pre-trained weights on ImageNet
model_vgg16 = models.vgg16(pretrained=True)

# Cancelling gradient descent calculation for all the layers
for param in model_vgg16.parameters():
    param.requires_grad = False

# Initiate last three convolutional layers to train    
model_vgg16.features[24] = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
model_vgg16.features[25] = nn.ReLU(inplace=True)
model_vgg16.features[26] = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
model_vgg16.features[27] = nn.ReLU(inplace=True)
model_vgg16.features[28] = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
model_vgg16.features[29] = nn.ReLU(inplace=True)
model_vgg16.features[30] = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_vgg16.classifier[0].in_features
model_vgg16.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 1024), 
    nn.ReLU(), 
    nn.Dropout(p=0.2),
    nn.Linear(1024, 512), 
    nn.ReLU(), 
    nn.Dropout(p=0.2),
    nn.Linear(512, 128), 
    nn.ReLU(), 
    nn.Dropout(p=0.2),
    nn.Linear(128, 2)
)

# Loading the device to CUDA
model_vgg16.to(device)

# Loss Function definition
criterion = nn.CrossEntropyLoss()

# Using Adam as the optimizer for the feature network
optimizer_feature = optim.Adam(model_vgg16.features.parameters(), lr=0.003)
# Using Stochastic Gradient Descent as the optimizer for the classifier network
optimizer_classifier = optim.SGD(model_vgg16.classifier.parameters(), lr=0.003, momentum=0.9)

# Decay LR by a factor of 0.1 every 4 epochs
feature_lr_scheduler = lr_scheduler.StepLR(optimizer_feature, step_size=4, gamma=0.1)
classifier_lr_scheduler = lr_scheduler.StepLR(optimizer_classifier, step_size=4, gamma=0.1)

In [ ]:
def train_model(model, criterion, optimizer_fe, optimizer_cl, scheduler1, scheduler2, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler1.step()
                scheduler2.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataset_loaders[phase]:
                # get the inputs
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                # zero the parameter gradients
                optimizer_fe.zero_grad()
                optimizer_cl.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer_fe.step()
                    optimizer_cl.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataset[phase])
            epoch_acc = running_corrects.item() / len(dataset[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_loss, validation_loss, train_accuarcy, validation_accuracy

In [ ]:
def evaluate_model(model, datalaoder, criterion):
    model.train(False)
    running_loss, running_corrects = 0, 0
    for data in datalaoder:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    test_loss = running_loss / len(dataset['test'])
    test_acc = running_corrects.item() / len(dataset['test'])
    print('Test Loss: {:.4f} Acc: {:.4f}'.format(test_loss, test_acc))
    return test_loss, test_acc

In [ ]:
model_ft, train_loss, valid_loss, train_accuracy, valid_accuracy = train_model(model_vgg16, criterion, optimizer_feature, optimizer_classifier, feature_lr_scheduler, classifier_lr_scheduler,
                       num_epochs=10)
# Free up Cached memory
# torch.cuda.empty_cache() 

In [ ]:
# Free up Cached memory
torch.cuda.empty_cache()

In [ ]:
# Save model
torch.save(model_ft, r"E:\Class_Notes_Sem2\ADM\Project\malaria_level_detection\first_model.pth")